In [1]:
using Laplacians

In [2]:
include("/Users/spielman/Laplacians/src/flowUtils.jl")

cutCapacity

In [3]:
include("/Users/spielman/Laplacians/src/min_cost_flow.jl")
#include("/Users/spielman/Laplacians/src/primalDualIPM.jl")
#include("/Users/spielman/Laplacians/src/max_flow_IPM.jl")

makeAdj (generic function with 1 method)

We introduce a data type for minimum cost flow problems.
It has an edge list, a vector of capacities of edges, a vector of costs, and a vector of demands.
We test this with the graph on 4 vertices with source 1 and destination 4.
We try to flow 1 unit of flow.  
The capacities on all edges are 0.7.
The costs on route 1-2-4 are 1, and on route 1-3-4 are 2.
So, it should flow most on route 1-2-4, as it does.

In [4]:
edges = [1 2; 2 4; 1 3; 3 4]
caps = ones(4)*0.7
costs = [1.0; 1; 2 ; 2]
dems = [1.0;0;0;-1]

mcfp = MCFproblem(edges, caps, costs, dems)

MCFproblem{Float64,Int64}([1 2; 2 4; 1 3; 3 4],[0.7,0.7,0.7,0.7],[1.0,1.0,2.0,2.0],[1.0,0.0,0.0,-1.0])

In [5]:
sol = min_cost_flow(mcfp)
flow = sol[1]

Iteration 1, ||r_p||=0.094281, ||r_d||=4.626043, mu=2.380000
Iteration 2, ||r_p||=0.005625, ||r_d||=0.276011, mu=0.160090
Iteration 3, ||r_p||=0.000521, ||r_d||=0.025576, mu=0.035827
Iteration 4, ||r_p||=0.000005, ||r_d||=0.000256, mu=0.000358
Iteration 5, ||r_p||=0.000000, ||r_d||=0.000003, mu=0.000004
Iteration 6, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000000
Termination tolerance reached.


4-element Array{Float64,1}:
 0.7
 0.7
 0.3
 0.3

The following cell computes the costs and check that the demands are satisfied.

In [6]:
reportMCFresults(mcfp,flow)

Cost: 2.599999998893948
Min flow: 0.29999999981555786
Min slack: 1.8414170188663093e-10
Error on demands: 7.371676602474508e-10


Now, let's try computing this using JuMP, an optimization package for Julia.

In [7]:
using JuMP
using Clp

In [8]:
function MCFjump(mcfp::MCFproblem)
    edge_list = mcfp.edge_list
    m = size(edge_list,1)
    n = maximum(edge_list)
    B = sparse(collect(1:m), edge_list[:,1], 1.0, m, n) -
      sparse(collect(1:m), edge_list[:,2], 1.0, m, n)
 

    mod = Model(solver=ClpSolver())
    @variable(mod, x[1:m] >= 0)
    @constraint(mod, x .<= mcfp.capacities)
    @constraint(mod, B'*x .== mcfp.demands)
    @objective(mod, Min, (mcfp.costs'*x)[1])
    @time status = solve(mod)

    f = getvalue(x)
    return f
    
end

MCFjump (generic function with 1 method)

In [9]:
f = MCFjump(mcfp)
reportMCFresults(mcfp,f)

  1.343271 seconds (143.89 k allocations: 6.436 MB, 0.54% gc time)
Cost: 2.6
Min flow: 0.30000000000000004
Min slack: 0.0
Error on demands: 0.0


We would like to compare the performance of our code to standard codes on some benchmark examples.  These benchmark examples usually come in the Dimacs format.  
In this notebook, we use the example "goto_8_08a.min"
from <a href="http://lime.cs.elte.hu/~kpeter/data/mcf/goto/">http://lime.cs.elte.hu/~kpeter/data/mcf/goto/</a>.

In [4]:
mcfp = readDimacsMCF("/Users/spielman/tmp/goto_8_08a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 180 216; 216 252],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0],[55614.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,-55614.0,0.0,0.0,0.0,0.0])

In [20]:
#sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-12)), tol=1e-2)
@time sol = min_cost_flow(mcfp, lapSolver = cholLap, tol = 1e-5)
flow = sol[1]
reportMCFresults(mcfp,flow)

Iteration 1, ||r_p||=82578.653557, ||r_d||=1092356.429364, mu=629220336.199898
Iteration 2, ||r_p||=82564.840913, ||r_d||=1092173.714707, mu=629298827.556181
Iteration 3, ||r_p||=82535.269542, ||r_d||=1091782.542463, mu=629463964.755363
Iteration 4, ||r_p||=82436.875911, ||r_d||=1090480.984355, mu=629993065.063792
Iteration 5, ||r_p||=82134.463833, ||r_d||=1086480.655419, mu=631435849.981872
Iteration 6, ||r_p||=81100.918447, ||r_d||=1072808.841952, mu=635028176.950339
Iteration 7, ||r_p||=77271.165564, ||r_d||=1022148.592548, mu=640629738.277026
Iteration 8, ||r_p||=61444.298970, ||r_d||=812789.651283, mu=625107449.020012
Iteration 9, ||r_p||=24622.141586, ||r_d||=325703.477930, mu=426319487.000312
Iteration 10, ||r_p||=10954.931848, ||r_d||=144912.634462, mu=288125231.811432
Iteration 11, ||r_p||=912.478442, ||r_d||=12070.331132, mu=28295686.974228
Iteration 12, ||r_p||=72.245337, ||r_d||=955.666572, mu=2616627.991254
Iteration 13, ||r_p||=34.953747, ||r_d||=462.370699, mu=2395909.66

In [7]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-12, verbose=true)), tol=1e-5)
flow = sol[1]
reportMCFresults(mcfp,flow)

Iteration 1, ||r_p||=82578.653557, ||r_d||=1092356.429364, mu=629220336.199898
Solver build time: 0.006 seconds.
PCG BLAS stopped after: 0.001 seconds and 45 iterations with relative error 9.072708175287244e-13.
PCG BLAS stopped after: 0.002 seconds and 45 iterations with relative error 7.629099585825e-13.
Iteration 2, ||r_p||=82564.840913, ||r_d||=1092173.714707, mu=629298827.556181
Solver build time: 0.003 seconds.
PCG BLAS stopped after: 0.002 seconds and 42 iterations with relative error 9.130551191746629e-13.
PCG BLAS stopped after: 0.001 seconds and 42 iterations with relative error 7.559846647594031e-13.
Iteration 3, ||r_p||=82535.269542, ||r_d||=1091782.542463, mu=629463964.755363
Solver build time: 0.002 seconds.
PCG BLAS stopped after: 0.002 seconds and 48 iterations with relative error 8.067710439830501e-13.
PCG BLAS stopped after: 0.002 seconds and 48 iterations with relative error 8.095117276207693e-13.
Iteration 4, ||r_p||=82436.875911, ||r_d||=1090480.984355, mu=62999306

As a sanity check, we compare the results from MCFjump.

In [22]:
f = MCFjump(mcfp)
mcfp.costs'*f

  0.011799 seconds (104 allocations: 674.000 KB)


1-element Array{Float64,1}:
 5.60871e8

This is very similar.  But, JuMP computed a flow of a slightly lower cost.
This should not be surprising, because we haven't figured out exactly what tol we want.
It might help to know that all the numbers in the Dimacs problems are integers.

In [23]:
mcfp.costs'*f - mcfp.costs'*flow

1-element Array{Float64,1}:
 -0.000637293

For comparison, we should also try the code from "Lemon", available at

<a href="http://lemon.cs.elte.hu/trac/lemon">http://lemon.cs.elte.hu/trac/lemon</a>

It was easy to install on my Mac (I just needed cmake, which I got by `brew install cmake`)

Here is a transcript of running it:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_08a.min 
Problem type: min
Num of nodes: 256
Num of arcs:  2048

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00467205s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 7.70092e-05s
Run NetworkSimplex: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00143886s

Feasible flow: found
Min flow cost: 560870539
```

## More Tests

Let's just do more tests on the GOTO problems, just to get a feel for the speeds involved.  Keep in mind that different problems will have very different run times, and that our code can be sped up a lot.

The Laplacian systems generated by the IPMs can have edge weights varying by 12 orders of magnitude!  This causes some of the solvers to run into numerical issues.  This is interesting to me, as it points out a place where our solvers need more development.

In [8]:
mcfp = readDimacsMCF("/Users/spielman/tmp/goto_8_13a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 7362 7771; 7771 8180],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0],[148449.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [25]:
flow = MCFjump(mcfp)
reportMCFresults(mcfp, flow)

LoadError: InterruptException:

In [26]:
@time sol = min_cost_flow(mcfp, lapSolver = cholLap, tol = 1e-5)
flow = sol[1]
reportMCFresults(mcfp,flow)

Iteration 1, ||r_p||=453081.641457, ||r_d||=17895730.774237, mu=4243561764.927625
Iteration 2, ||r_p||=453052.371187, ||r_d||=17894574.662779, mu=4243766556.176835
Iteration 3, ||r_p||=452990.562639, ||r_d||=17892133.360744, mu=4244195683.393663
Iteration 4, ||r_p||=452786.206173, ||r_d||=17884061.728700, mu=4245590107.857270
Iteration 5, ||r_p||=452180.033144, ||r_d||=17860119.223978, mu=4249556361.298622
Iteration 6, ||r_p||=450269.943470, ||r_d||=17784674.872633, mu=4260890043.963723
Iteration 7, ||r_p||=444097.899954, ||r_d||=17540892.695234, mu=4290484655.441635
Iteration 8, ||r_p||=423190.765057, ||r_d||=16715106.737154, mu=4356806296.956453
Iteration 9, ||r_p||=346884.506327, ||r_d||=13701176.933601, mu=4416879841.740283
Iteration 10, ||r_p||=165881.623160, ||r_d||=6551960.169735, mu=4473868869.235403
Iteration 11, ||r_p||=96370.427357, ||r_d||=3806420.443424, mu=3795576973.591908
Iteration 12, ||r_p||=61757.597106, ||r_d||=2439289.589225, mu=3135266450.305884
Iteration 13, ||r_

In [10]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-8, verbose=true, maxits=1000)), tol=1e-6)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||=453081.641457, ||r_d||=17895730.774237, mu=4243561764.927625
Solver build time: 0.209 seconds.
PCG BLAS stopped after: 0.043 seconds and 60 iterations with relative error 9.411970164906377e-9.
PCG BLAS stopped after: 0.037 seconds and 55 iterations with relative error 8.97352677945455e-9.
Iteration 2, ||r_p||=453052.371187, ||r_d||=17894574.662778, mu=4243766556.176805
Solver build time: 0.183 seconds.
PCG BLAS stopped after: 0.038 seconds and 58 iterations with relative error 9.710007652894914e-9.
PCG BLAS stopped after: 0.034 seconds and 53 iterations with relative error 9.601991714128221e-9.
Iteration 3, ||r_p||=452990.562639, ||r_d||=17892133.360744, mu=4244195683.393672
Solver build time: 0.113 seconds.
PCG BLAS stopped after: 0.039 seconds and 55 iterations with relative error 9.113107005534526e-9.
PCG BLAS stopped after: 0.028 seconds and 51 iterations with relative error 9.182274085479862e-9.
Iteration 4, ||r_p||=452786.206172, ||r_d||=17884061.728700, mu=4

LoadError: BoundsError: attempt to access 0-element Array{Int64,1} at index [0]

Here is what Lemon did:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_13a.min 
Problem type: min
Num of nodes: 8192
Num of arcs:  65536

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.08s, s: 0s, cu: 0s, cs: 0s, real: 0.0842741s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00165105s
Run NetworkSimplex: u: 0.43s, s: 0s, cu: 0s, cs: 0s, real: 0.439616s

Feasible flow: found
Min flow cost: 18217956686
```

It was absurdly faster.  But, I won't worry about that yet.


Here is how Goldberg's CS2 code does, 
which I obtained from

<a href="https://github.com/iveney/cs2">https://github.com/iveney/cs2</a>

```
Dans17:cs2 spielman$ time ./cs2 < ~/tmp/goto_8_13a.min > out.txt
warning: this program uses gets(), which is unsafe.

real	0m2.425s
user	0m2.344s
sys	0m0.027s

Dans17:cs2 spielman$ head out.txt 
c CS 4.6
c Commercial use requires a licence
c contact igsys@eclipse.net
c
c nodes:            8192     arcs:            65536
c scale-factor:       12     cut-off-factor:   79.1
c
c time:             2.25     cost:      18217956686
c refines:             4     discharges:    3409417
c pushes:        3621130     relabels:      1381573
```

## goto_8_14b.min

In [28]:
mcfp = readDimacsMCF("/Users/spielman/tmp/goto_8_14b.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 15120 15750; 15750 16380],[2481.0,951.0,346.0,61.0,8368.0,9891.0,6933.0,783.0,155.0,37.0  …  202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0],[807.0,189.0,40.0,285.0,4.0,5.0,323.0,675.0,464.0,19.0  …  38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0],[202084.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,-202084.0,0.0,0.0,0.0,0.0])

Some of these codes run into numerical issues at very low tolerances.  So, I will be more generous for this run.

In [30]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-10)), tol=1e-1)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||=710335.242861, ||r_d||=34926531.711984, mu=7807296078.547482
Iteration 2, ||r_p||=710300.793288, ||r_d||=34924837.858103, mu=7807574079.255902
Iteration 3, ||r_p||=710228.138775, ||r_d||=34921265.502375, mu=7808157173.384199
Iteration 4, ||r_p||=709988.349715, ||r_d||=34909475.294452, mu=7810056574.101986
Iteration 5, ||r_p||=709279.297255, ||r_d||=34874611.836040, mu=7815479029.834824
Iteration 6, ||r_p||=707052.845839, ||r_d||=34765139.264034, mu=7831036282.128851
Iteration 7, ||r_p||=699906.432874, ||r_d||=34413756.699882, mu=7872079861.979634
Iteration 8, ||r_p||=676071.439605, ||r_d||=33241811.964461, mu=7967830375.013041
Iteration 9, ||r_p||=592960.715561, ||r_d||=29155333.969612, mu=8109347980.602172
Iteration 10, ||r_p||=351006.716477, ||r_d||=17258677.979666, mu=7482254378.864234
Iteration 11, ||r_p||=202423.501500, ||r_d||=9952977.717779, mu=7719210299.084393
Iteration 12, ||r_p||=90307.241862, ||r_d||=4440324.168605, mu=4239312215.937900
Iteration 13, ||

In [33]:
@time sol = min_cost_flow(mcfp, lapSolver = cholLap , tol=1)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||=710335.242861, ||r_d||=34926531.711984, mu=7807296078.547482
Iteration 2, ||r_p||=710300.793288, ||r_d||=34924837.858103, mu=7807574079.255902
Iteration 3, ||r_p||=710228.138775, ||r_d||=34921265.502375, mu=7808157173.384199
Iteration 4, ||r_p||=709988.349715, ||r_d||=34909475.294452, mu=7810056574.101980
Iteration 5, ||r_p||=709279.297255, ||r_d||=34874611.836040, mu=7815479029.834822
Iteration 6, ||r_p||=707052.845838, ||r_d||=34765139.264034, mu=7831036282.128863
Iteration 7, ||r_p||=699906.432874, ||r_d||=34413756.699882, mu=7872079861.979658
Iteration 8, ||r_p||=676071.439605, ||r_d||=33241811.964460, mu=7967830375.012883
Iteration 9, ||r_p||=592960.715561, ||r_d||=29155333.969602, mu=8109347980.599541
Iteration 10, ||r_p||=351006.716479, ||r_d||=17258677.979778, mu=7482254378.860389
Iteration 11, ||r_p||=202423.501497, ||r_d||=9952977.717665, mu=7719210299.076655
Iteration 12, ||r_p||=90307.241860, ||r_d||=4440324.168532, mu=4239312215.859472
Iteration 13, ||

If I try tol = 1e-4, that solver dies due to numerical issues.  

And, here is Lemon:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_14b.min 
Problem type: min
Num of nodes: 16384
Num of arcs:  131072

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.18s, s: 0s, cu: 0s, cs: 0s, real: 0.187166s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00391507s
Run NetworkSimplex: u: 1.6s, s: 0.01s, cu: 0s, cs: 0s, real: 1.61133s

Feasible flow: found
Min flow cost: 41834525746
```


And, here is CS2:

```
Dans17:cs2 spielman$ time ./cs2 < ~/tmp/goto_8_14b.min > out.txt
warning: this program uses gets(), which is unsafe.

real	0m22.478s
user	0m21.870s
sys	0m0.096s
Dans17:cs2 spielman$ head out.txt 
c CS 4.6
c Commercial use requires a licence
c contact igsys@eclipse.net
c
c nodes:           16384     arcs:           131072
c scale-factor:       12     cut-off-factor:  107.3
c
c time:            21.70     cost:      41834525746
c refines:             5     discharges:    9131788
c pushes:        9571379     relabels:      3496218
```

## goto_8_16a
Lemon (which is implementing a network simplex code) starts to scale badly when given this problem, as does CS2.
Our scales as we would expect.
This problem is 4 times bigger than `goto_8_14b`.

I stopped CS2 after 7 minutes, because it was driving me nuts.
Here is how Lemon did:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_16a.min 
Problem type: min
Num of nodes: 65536
Num of arcs:  524288

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.68s, s: 0.01s, cu: 0s, cs: 0s, real: 0.711415s
Setup NetworkSimplex class: u: 0.01s, s: 0.01s, cu: 0s, cs: 0s, real: 0.015645s
Run NetworkSimplex: u: 80.66s, s: 0.26s, cu: 0s, cs: 0s, real: 81.4035s
```

In [4]:
mcfp = readDimacsMCF("/Users/spielman/tmp/goto_8_16a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 62244 63882; 63882 65520],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0],[331827.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [36]:
@time sol = min_cost_flow(mcfp, lapSolver = cholLap, tol=1)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||=1370095.345474, ||r_d||=116864630.148304, mu=20862256364.106258
Iteration 2, ||r_p||=1370054.044492, ||r_d||=116861107.310279, mu=20862716994.257442
Iteration 3, ||r_p||=1369966.873185, ||r_d||=116853671.884245, mu=20863680411.510319
Iteration 4, ||r_p||=1369679.984777, ||r_d||=116829201.245920, mu=20866833498.161743
Iteration 5, ||r_p||=1368835.380943, ||r_d||=116757159.314662, mu=20875887167.123146
Iteration 6, ||r_p||=1366196.332895, ||r_d||=116532057.189363, mu=20902067158.667389
Iteration 7, ||r_p||=1357784.805709, ||r_d||=115814581.564949, mu=20972179454.468510
Iteration 8, ||r_p||=1330063.502768, ||r_d||=113450045.530183, mu=21142627083.457748
Iteration 9, ||r_p||=1236606.194701, ||r_d||=105478444.299676, mu=21468228709.443768
Iteration 10, ||r_p||=897148.187194, ||r_d||=76523791.888600, mu=21175505322.016315
Iteration 11, ||r_p||=555327.863769, ||r_d||=47367641.693511, mu=23208513306.810078
Iteration 12, ||r_p||=294413.760916, ||r_d||=25112526.214785, mu=16

In [37]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-10)), tol=1)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||=1370095.345474, ||r_d||=116864630.148304, mu=20862256364.106258
Iteration 2, ||r_p||=1370054.044492, ||r_d||=116861107.310279, mu=20862716994.257439
Iteration 3, ||r_p||=1369966.873185, ||r_d||=116853671.884245, mu=20863680411.510315
Iteration 4, ||r_p||=1369679.984777, ||r_d||=116829201.245920, mu=20866833498.161751
Iteration 5, ||r_p||=1368835.380943, ||r_d||=116757159.314662, mu=20875887167.123150
Iteration 6, ||r_p||=1366196.332895, ||r_d||=116532057.189363, mu=20902067158.667427
Iteration 7, ||r_p||=1357784.805709, ||r_d||=115814581.564951, mu=20972179454.468803
Iteration 8, ||r_p||=1330063.502768, ||r_d||=113450045.530191, mu=21142627083.458965
Iteration 9, ||r_p||=1236606.194702, ||r_d||=105478444.299744, mu=21468228709.455700
Iteration 10, ||r_p||=897148.187197, ||r_d||=76523791.888869, mu=21175505322.071014
Iteration 11, ||r_p||=555327.863778, ||r_d||=47367641.694227, mu=23208513306.774761
Iteration 12, ||r_p||=294413.760965, ||r_d||=25112526.218994, mu=16

In [8]:
include("/Users/spielman/Laplacians/src/matlabSolvers.jl")

>> >> >> Connected to Matlab!


matlabCmgLap

In [9]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->matlabCmgLap(h, tol=1e-8)), tol=1)
flow = sol[1]

Iteration 1, ||r_p||=1370095.345474, ||r_d||=116864630.148304, mu=20862256364.106258
>> >> >> 

Level 2. Graph vertices: 21992 
Density : 21.3993
Reduction factor in vertices : 2.97995
Reduction factor in edges    : 2.08458
*****************
Level 3. Graph vertices: 7408 
Density : 29.8799
Reduction factor in vertices : 2.96868
Reduction factor in edges    : 2.12611
*****************
Level 4. Graph vertices: 2408 
Density : 35.6944
Reduction factor in vertices : 3.07641
Reduction factor in edges    : 2.57527
*****************
Level 5. Graph vertices: 582 
Density : 33.8866
Reduction factor in vertices : 4.13746
Reduction factor in edges    : 4.35818
*****************
Level 6. Graph vertices: 198 
Density : 30.7475
Reduction factor in vertices : 2.93939
Reduction factor in edges    : 3.23949
*****************
Hierarchy constructed in 0.220000 seconds
>> >> >> 

Level 2. Graph vertices: 21992 
Density : 21.3993
Reduction factor in vertices : 2.97995
Reduction factor in edges    : 2.08458

524288-element Array{Float64,1}:
   8642.0      
    741.0      
    453.0      
     61.9999   
    340.993    
   1029.0      
   2640.0      
   1762.0      
    155.0      
     30.9999   
   9912.99     
   1070.0      
   9689.0      
      ⋮        
  75100.1      
  75095.1      
  83789.1      
  92948.1      
      1.01559e5
      1.06557e5
      1.0419e5 
 107967.0      
 120469.0      
 125774.0      
 133627.0      
 143580.0      

In [10]:
reportMCFresults(mcfp, flow)

Cost: 2.0566500432805237e11
Min flow: 1.1533763144431898e-7
Min slack: 1.1589416715196421e-7
Error on demands: 1.676119310230133
